In [1]:
import random
import re

In [2]:
ALPHABET = set('abcdefghijklmnopqrstuvwxyz')

def load_words():
    with open('words.txt') as f:
        valid_words = set(f.read().split())

    return valid_words

def filter_words(words):
    output = words.copy()
    for word in words:
        # remove triple letters
        for index, letter in enumerate(word[2:]):
            if (word[index] == word[index + 1] == letter) or letter not in ALPHABET:
                print(f'Removing \'{word}\'')
                output.remove(word)
                break
    
    return output

def check_words(words, *word_list):
    output = []
    for word in word_list:
        output.append(word in words)
    
    if len(output) == 1:
        return output[0]
    
    return output

def get_random(words, count):
    for i in range(count):
        return random.sample(list(words), count)

WORDS = filter_words(load_words())

Removing 'headmistressship'
Removing 'aaa'
Removing 'patronessship'
Removing 'brrr'
Removing 'baccillla'
Removing 'blottto'
Removing 'mmmm'
Removing 'bonnne'
Removing 'xviii'
Removing 'dipppier'
Removing 'asssembler'
Removing 'apppetible'
Removing 'zipppiest'
Removing 'demigoddessship'
Removing 'zipppier'
Removing 'passsaging'
Removing 'dipppiest'
Removing 'addda'
Removing 'wallless'
Removing 'outttore'
Removing 'bossship'
Removing 'bulllike'
Removing 'viii'
Removing 'xxx'
Removing 'dipppy'
Removing 'gulllike'
Removing 'xiii'
Removing 'bonnnes'
Removing 'xxiii'
Removing 'oooo'
Removing 'iii'
Removing 'whenceeer'
Removing 'outttorn'
Removing 'ieee'
Removing 'pasticcci'
Removing 'bimilllennia'
Removing 'goddessship'
Removing 'earlesss'


In [3]:
print(check_words(WORDS, 'hexadecimal', 'elucidate', 'perfunctory', 'fhqwhgads', 'eef', 'gnarf'))
print(get_random(WORDS, 10))

[True, True, True, False, False, False]
['amblychromatic', 'bioecologically', 'ocellus', 'bowpots', 'schoolboyishness', 'theatric', 'coaptation', 'dropberry', 'unasterisk', 'areole']


In [30]:
class Chunk:
    def __init__(self, text):
        self.text = text
    
class Word(Chunk):
    def __init__(self, text):
        super().__init__(text)
        self.reveal = ['_'] * len(text)
        self.remaining = len(text)

    def reveal_guess(self, guess):
        for index, letter in enumerate(self.text):
            if letter == guess:
                self.reveal[index] = letter
                self.remaining -= 1
    
    def match_regex(self, letter):
        pattern = ''.join('^' + map(lambda let: '.' if let == '_' else let, self.reveal) + '$')
        return re.compile(pattern)
    
    def solved(self):
        return self.remaining <= 0
    
    def __str__(self):
        return ''.join(self.reveal)
    
class Space(Chunk):
    def __init__(self, text):
        super().__init__(text)
    
    def reveal_guess(self, letter):
        pass
    
    def solved(self):
        return True
    
    def __str__(self):
        return self.text
    
class Solution:
    def parse_solution(words, solution):
        if '_' in solution:
            raise Exception('No underscores allowed.')

        solution = solution.strip().lower()
        found_words = re.split(PATTERN, solution)

        solution_words = []
        solution_revealed = []
        for word in found_words:
            if word.isnumeric():
                solution_words.append(Space(word))

            elif word.isalnum():
                if check_words(words, word):
                    solution_words.append(Word(word))
                else:
                    raise Exception('Solution includes non-word.')

            else:
                solution_words.append(Space(word))

        # check that there is at least one word
        for word in solution_words:
            if isinstance(word, Word):
                return solution_words

        raise Exception('No words found in solution.')
        
    def __init__(self, valid_words, solution):
        self.solution = Solution.parse_solution(valid_words, solution)
    
    def __iter__(self):
        return iter(self.solution)
    
    def __eq__(self, other):
        return isinstance(other, Solution) and self.solution == other.solution

class Hangman:
    # solution is a list of chunks
    def __init__(self, valid_words, solution):
        self.valid_words = valid_words
        self.solution = Solution(self.valid_words, solution)
        self.guessed = []
        self.guesses = 0
    
    def reveal_guess(self, guess):
        guess = guess.strip().lower()
        if len(guess) != 1:
            raise Exception('Guess must be one letter.')
        if not guess.isalpha():
            raise Exception('Guess must be a letter.')
        if guess in self.guessed:
            raise Exception('Letter already guessed.')
        
        for chunk in self.solution:
            chunk.reveal_guess(guess)
            
        self.guesses += 1
        self.guessed.append(guess)
            
    def solved(self):
        return all(chunk.solved() for chunk in self.solution)
        
    def print_reveal(self):
        for chunk in self.solution:
            print(chunk, end='')
        print(f'\tGuessed: {self.guessed}\n')
    
    def print_win(self):
        for chunk in self.solution:
            print(chunk.text, end='')
        print(f'\tGuessed: {self.guessed}\n')
        print(f'Congratulations! You solved the puzzle in {self.guesses} guesses.')

In [31]:
PATTERN = re.compile('(\W+)')

def get_hangman_solution_input():
    return input('Input a solution: ')

def get_hangman_guess_input(game):
    return input('Input a guess: ')

def play_hangman(words, generation_fn, guess_fn):
    game = None
    while game is None:
        try:
            game = Hangman(WORDS, generation_fn())
        except Exception as e:
            print(f'Error: {e}')

    while not game.solved():
        game.print_reveal()
        
        guessing = True
        while guessing:
            try:
                game.reveal_guess(guess_fn(game))
                guessing = False
            except Exception as e:
                print(f'Error: {e}')               
    
    game.print_win()
        
play_hangman(WORDS, get_hangman_solution_input, get_hangman_guess_input)

Input a solution: happy
_____	Guessed: []

Input a guess: a
_a___	Guessed: ['a']

Input a guess: p
_app_	Guessed: ['a', 'p']

Input a guess: y
_appy	Guessed: ['a', 'p', 'y']

Input a guess: h
happy	Guessed: ['a', 'p', 'y', 'h']

Congratulations! You solved the puzzle in 4 guesses.
